### Agrupando y agregando datos

Hasta ahora nos hemos centrado sobre todo en operaciones para manipular y transformar los datos contenidos en Series y DataFrames, en su conjunto o filtrando algunos valores.

Uno de los procedimientos de análisis de datos más común consiste en dividir nuestro conjunto de datos en grupos disjuntos en base a algún criterio o variables y realizar algún tipo de operación o análisis sobre cada grupo, como calcular estadísticas del grupo (valores medios, mínimos, máximos, etc.).

Este tipo de procesos sobre grupos de datos te resultará familiar si has trabajado con bases de datos, ya que es similar a las cláusulas tipo `GROUP BY` de `SQL`. Hadley Wickham (autor de multitud de paquetes en R para manejo y análisis de datos) se refiere a este tipo de procesos como "_split, apply, combine_" (divide, aplica y combina). En el mundo del _Big Data_ este modelo de computación es conocido como _map-reduce_.

Veamos un ejemplo gráfico. La siguiente figura ilustra este mecanismo _split, apply, combine_ para calcular la temperatura media de unas ciudades a partir de varias observaciones.

<img src="./img/fig_split_apply_combine.png" />

En Pandas, los objetos de tipo DataFrame incorporan el método `groupby()` para ejecutar estos procesos.

In [ ]:
# Volvamos a cargar los datos meteorológicos
meteo_mes = pd.read_csv("../U09_datasets/meteo_mes_agg.csv", sep = ";")

# Calculamos valores promedio agrupando por ciudad
meteo_mes.groupby('ciudad').mean()

,año,mes,temp_c,viento_vel_kmh
ciudad,,,,
Barcelona,2015.5,6.5,17.012500,14.250000
Bilbao,2015.5,6.5,15.404167,8.862500
La Coruña,2015.5,6.5,14.412500,10.437500
Madrid,2015.5,6.5,15.637500,10.079167
Malaga,2015.5,6.5,19.083333,11.495833
Sevilla,2015.5,6.5,19.183333,9.266667
Valencia,2015.5,6.5,18.275000,10.529167
Zaragoza,2015.5,6.5,15.429167,16.670833


En este ejemplo hemos agrupado las filas según la columna `ciudad` y después hemos pedido calcular los valores promedio para cada grupo. Como ves, la función `mean()` se ha aplicado a todas las columnas (en este caso, a las de tipo numérico). Este es el comportamiento por defecto cuando usamos funciones de agregación sobre el resultado de `df.groupby()`.

Podemos seleccionar las columnas sobre las que queremos aplicar la función, indexando como haríamos con un DataFrame.

In [ ]:
# Indexamos columnas sobre el resultado de `df.groupby()
# antes de aplicar la función
meteo_mes.groupby('ciudad')['temp_c'].mean()

ciudad
Barcelona    17.012500
Bilbao       15.404167
La Coruña    14.412500
Madrid       15.637500
Malaga       19.083333
Sevilla      19.183333
Valencia     18.275000
Zaragoza     15.429167
Name: temp_c, dtype: float64

Ell método `df.groupby()` también nos permite especificar varias columnas como criterio de agrupación. En este caso, utilizamos una lista con los nombres de las columnas para agrupar.

In [ ]:
# Valor promedio agrupando por ciudad y año
meteo_mes.groupby(['ciudad','año'])['temp_c'].mean().head()

ciudad     año 
Barcelona  2015    17.033333
           2016    16.991667
Bilbao     2015    15.608333
           2016    15.200000
La Coruña  2015    14.808333
Name: temp_c, dtype: float64

¿Te has fijado? Al agrupar por varias columnas, tenemos un índice jerárquico para las filas.

In [ ]:
meteo_mes.groupby(['ciudad','año'])['temp_c'].mean().index

MultiIndex(levels=[['Barcelona', 'Bilbao', 'La Coruña', 'Madrid', 'Malaga', 'Sevilla', 'Valencia', 'Zaragoza'], [2015, 2016]],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7], [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]],
           names=['ciudad', 'año'])

El método `df.groupby()` devuelve un objeto de tipo `DataFrameGroupBy`. A efectos prácticos, podemos considerarlo como una colección de DataFrames, uno por grupo creado.

In [ ]:
meteo_mes.groupby('ciudad')

Sobre los grupos resultantes podemos aplicar directamente cualquiera de las funciones de agregación y  estadísticas que ya te hemos presentado.

Pero hay mucho más. Disponemos de varios métodos para realizar distintos tipos de operaciones sobre cada grupo.

#### Agregados sobre grupos

Empecemos por el método `aggregate()`. Este método nos permite calcular múltiples valores agregados de forma simultánea, indicando en una lista las funciones de agregación a utilizar (ya sean proporcionadas por la librería o definidas por el usuario).

In [ ]:
# Agrupamos por ciudad
# y calculamos los valores de media y mediana
# de temperatura y velocidad de viento para cada ciudad
meteo_mes.groupby('ciudad')['temp_c','viento_vel_kmh'].aggregate(['mean', np.median, lambda x: min(x)])

temp_c                 viento_vel_kmh                
                mean median <lambda>           mean median <lambda>
ciudad                                                             
Barcelona  17.012500  16.00      9.0      14.250000  14.45      5.0
Bilbao     15.404167  15.90      8.0       8.862500   8.70      6.1
La Coruña  14.412500  14.45      9.3      10.437500  10.20      5.8
Madrid     15.637500  14.85      4.4      10.079167  10.35      5.4
Malaga     19.083333  18.20     11.4      11.495833  11.20      8.5
Sevilla    19.183333  18.75      9.6       9.266667   9.00      6.4
Valencia   18.275000  17.65     10.1      10.529167  10.00      5.0
Zaragoza   15.429167  15.15      6.0      16.670833  17.45      5.9

¿Has prestado atención cómo hemos usado el método `aggregate()`? Este ejemplo te muestra distintas formas de especificar las funciones de agregación a aplicar. En el primer elemento hemos escrito una cadena de texto para referirnos a la media. Pandas permite hacer esto con las operaciones comunes incluidas, él se encarga de traducir la cadena de texto a la función correspondiente. En el segundo elemento hemos utilizado directamente el nombre de una función y no una cadena de texto (en este caso el método para calcular la mediana de la librería NumPy). Y en el último elemento incluimos una expresión _lambda_ a medida (también podríamos haber puesto el nombre de una función definida por nosotros.

¿Qué me dices del resultado? Ha creado un índice jerárquico para las columnas. En el primer nivel tienes los nombres de las columnas originales y en el segundo nivel el nombre de los agregados que hemos calculado.